In [50]:
apikey = "7RA6dB7O1NBSWBjmFGuP4DlIZP8DeWqeHW3Rrkr8"
import fooddatacentral as fdc
import pandas as pd
import requests
import time

df = pd.read_csv("trader_joes - trader_joes.csv")


for idx, row in df.iterrows():
    fdc_id = row["fdc_id"]

data_list = []

for fdcid in df["fdc_id"]:
    try:
        nutrient = fdc.nutrients(apikey, fdc_id=fdcid)

        cols = [f"{idx} ({row['Units']})" for idx, row in nutrient.iterrows()]
        data = dict(zip(cols, nutrient["Quantity"].values))
        
        data["fdc_id"] = fdcid
        data_list.append(data)
    except Exception as e:
        # print(f"Failed on FDC ID {fdcid}, error: {e}")

        continue

df2 = pd.DataFrame(data_list)
df2.to_csv("samsung_galaxy_ultra_nutritional_data.csv", index=False)


In [69]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

nutrition = pd.read_csv("samsung_galaxy_ultra_nutritional_data.csv")
minimum_req = pd.read_csv("minimum_req.csv")
ingredients = pd.read_csv("trader_joes_enriched.csv")
shitty = pd.read_csv("USDA-305tj(Sheet1).csv")

nutrition_with_upc = nutrition.merge(ingredients[['fdc_id', 'gtin_upc']], on='fdc_id', how='left')
shitty_renamed = shitty.rename(columns={"GTIN/UPC": "gtin_upc"})
nutrition_with_prices = nutrition_with_upc.merge(
    shitty_renamed[['gtin_upc', 'Price', 'Name']], 
    on='gtin_upc',
    how='left'
)

nutrition_with_prices.dropna(subset=['Price'], inplace=True)
nutrients_to_optimize = minimum_req['Nutrition'].values
available_nutrients = [col for col in nutrition_with_prices.columns if col.split(" (")[0] in nutrients_to_optimize]
costs = nutrition_with_prices['Price'].values
A = nutrition_with_prices[available_nutrients].fillna(0).values.T  
b = minimum_req.set_index('Nutrition').loc[ [col.split(" (")[0] for col in available_nutrients], "F 19-30"].apply(pd.to_numeric, errors='coerce').fillna(0).values.flatten()

bounds = [(0, None)] * len(costs) 


print(f"A_ub shape: {A.shape}") 
print(f"b_ub shape: {b.shape}") 
if len(b.shape) > 1: 
    print("b_ub is not 1D! Flattening again...") 
    b = b.flatten()

if A.shape[0] != b.shape[0]: 
    if A.shape[0] > b.shape[0]: 
        b = np.pad(b, (0, A.shape[0] - b.shape[0]), mode='constant', constant_values=0) 
    else: 
        b = b[:A.shape[0]]

result = linprog(c=costs, A_ub=-A, b_ub=-b, bounds=bounds, method='highs')

if result.success:
    nutrition_with_prices['Optimal Quantity'] = result.x
    selected_foods = nutrition_with_prices[nutrition_with_prices['Optimal Quantity'] > 0]
    solution_df = selected_foods[['fdc_id', 'gtin_upc', 'Name', 'Price', 'Optimal Quantity']]
    total_cost = result.fun  
else:
    solution_df = pd.DataFrame({"Error": ["Optimization Failed"]})
    total_cost = None


print(solution_df)

if total_cost is not None:
    print(f"Minimum Total Cost: ${total_cost:.2f}")

A_ub shape: (16, 673)
b_ub shape: (16,)
      fdc_id  gtin_upc                                     Name  Price  \
40    373612    180245        FROSTED SHREDDED BITE SIZE WHEATS   2.99   
75    397403    507653  TRADER JOE'S, SMALL WHOLE GRAIN LENTILS   0.99   
133   403399    537698                   HEMP SEED, RAW SHELLED   6.49   
207  1849873    954174                  POTATO CHIPS, OLIVE OIL   2.99   
308  1171477    930895                       TACO SEASONING MIX   0.99   

     Optimal Quantity  
40           1.456058  
75           0.128323  
133          0.380655  
207          2.272513  
308          2.926415  
Minimum Total Cost: $16.64


In [51]:
df = pd.read_csv("samsung_galaxy_ultra_nutritional_data.csv")
print(df.columns)

Index(['Vitamin A, IU (IU)', 'Iron, Fe (mg)', 'Total lipid (fat) (g)',
       'Vitamin C, total ascorbic acid (mg)', 'Fiber, total dietary (g)',
       'Fatty acids, total saturated (g)', 'Calcium, Ca (mg)',
       'Vitamin D (D2 + D3), International Units (IU)', 'Protein (g)',
       'Cholesterol (mg)', 'Sodium, Na (mg)', 'Energy (kcal)',
       'Fatty acids, total trans (g)', 'Total Sugars (g)',
       'Carbohydrate, by difference (g)', 'fdc_id',
       'Fatty acids, total polyunsaturated (g)',
       'Fatty acids, total monounsaturated (g)', 'Potassium, K (mg)',
       'Fiber, insoluble (g)', 'Fiber, soluble (g)', 'Thiamin (mg)',
       'Vitamin B-12 (µg)', 'Selenium, Se (µg)', 'Vitamin B-6 (mg)',
       'Folate, total (µg)', 'Riboflavin (mg)', 'Niacin (mg)',
       'Copper, Cu (mg)', 'Zinc, Zn (mg)', 'Magnesium, Mg (mg)',
       'Phosphorus, P (mg)', 'Manganese, Mn (mg)', 'Vitamin A (µg)'],
      dtype='object')


In [6]:
df_req = pd.read_csv("minimum_req.csv")
df_foods = pd.read_csv("trader_joes_enriched.csv")

name_map = {
    'Energy': 'calories_kcal',
    'Protein': 'protein_g',
    'Fiber, total dietary': 'fiber_g',
    'Fiber': 'fiber_g', 
    'Carbohydrate, by difference': 'carb_g',
    'Calcium, Ca': 'calcium_mg',
    'Iron, Fe': 'iron_mg',
    'Magnesium, Mg': 'magnesium_mg',
    'Niacin': 'niacin_mg',
    'Phosphorus, P': 'phosphorus_mg',
    'Potassium, K': 'potassium_mg',
    'Riboflavin': 'riboflavin_mg',
    'Thiamin': 'thiamin_mg',
    'Vitamin A, RAE': 'vita_rae_mcg',
    'Vitamin B-12': 'vitb12_mcg',
    'Vitamin B-6': 'vitb6_mg',
    'Vitamin C, total ascorbic acid': 'vitc_mg',
    'vitamin C, total ascorbic acid': 'vitc_mg', 
    'vitamin E (alpha-tocopherol)': 'vite_mg',
    'Vitamin K (phylloquinone)': 'vitk_mcg',
    'Zinc, Zn': 'zinc_mg'
}

df_req['Nutrition_renamed'] = df_req['Nutrition'].map(name_map)

def get_rda_value(nutrient_col, group, df_req):
    row_match = df_req[df_req['Nutrition_renamed'] == nutrient_col]
    if len(row_match) == 0:
        return None
    return row_match[group].values[0]

group_name = "F19-30"

for original_name, food_column in name_map.items():
    if food_column not in df_foods.columns:
        continue  
    
    rda_val = get_rda_value(food_column, group_name, df_req)
    
    if (rda_val is None) or (rda_val == 0):
        continue
    
    pct_col = f"pct_{group_name}_{food_column}" 
    df_foods[pct_col] = (df_foods[food_column] / rda_val) * 100

df_foods.to_csv("trader_joes_with_rda_percentages.csv", index=False)

In [8]:
id = 356707 # Put an FDC ID HERE!
print(type(fdc.nutrients(apikey,fdc_id=id)))

<class 'pandas.core.frame.DataFrame'>
